In [13]:
import pandas as pd
import folium
from folium.plugins import HeatMap
from branca.element import Template, MacroElement

In [14]:
# CSV-Datei laden
file_path = "../output/ScrapedJobOffersAllCleaned.csv"
df = pd.read_csv(file_path)

In [15]:
# Relevante Rollen erkennen
def extract_data_role(title):
    title = str(title).lower()
    if "analyst" in title:
        return "Data Analyst"
    elif "engineer" in title:
        return "Data Engineer"
    elif "scientist" in title:
        return "Data Scientist"
    return None

df['data_role'] = df['job_title'].apply(extract_data_role)
df_filtered = df[df['data_role'].notnull()].copy()

In [16]:
job_counts = df_filtered.groupby('city').size().reset_index(name='count')

# Nur Österreich-Städte
austria_cities = ['Wien', 'Linz', 'Graz', 'Salzburg', 'Innsbruck']
austria_jobs = job_counts[job_counts['city'].isin(austria_cities)].copy()

In [17]:
# Feste Koordinaten für Städte
city_coords = {
    "Wien": (48.2082, 16.3738),
    "Linz": (48.3069, 14.2858),
    "Graz": (47.0707, 15.4395),
    "Salzburg": (47.8095, 13.0550),
    "Innsbruck": (47.2682, 11.3923),
}

# Koordinaten zuordnen
austria_jobs["lat"] = austria_jobs["city"].map(lambda c: city_coords[c][0])
austria_jobs["lon"] = austria_jobs["city"].map(lambda c: city_coords[c][1])


In [18]:
# HeatMap-Daten vorbereiten
heat_data = [
    [row['lat'], row['lon'], row['count']]
    for _, row in austria_jobs.iterrows()
]

# Karte erstellen
m = folium.Map(location=[47.5, 14.5], zoom_start=7, tiles="CartoDB dark_matter")

# Heatmap einfügen
HeatMap(
    heat_data,
    radius=35,  # Größerer Radius = weniger punktuell
    blur=20,  # Weichere Übergänge
    max_zoom=10,
    min_opacity=0.4  # Sichtbarer bei geringeren Werten
).add_to(m)


In [19]:
# HTML/CSS-Legende einfügen
legend_html = """
{% macro html(this, kwargs) %}
<div style="
    position: fixed;
    bottom: 40px; left: 40px; width: 160px; height: 120px;
    z-index:9999; font-size:14px;
    background-color: rgba(0, 0, 0, 0.6);
    padding: 10px;
    border-radius: 5px;
    color: white;
">
<b>Heatmap-Legende</b><br>
<span style="background: #ffffb2; width: 15px; height: 15px; display:inline-block;"></span> Wenige Jobs<br>
<span style="background: #fecc5c; width: 15px; height: 15px; display:inline-block;"></span> Mittel<br>
<span style="background: #fd8d3c; width: 15px; height: 15px; display:inline-block;"></span> Viele Jobs<br>
<span style="background: #e31a1c; width: 15px; height: 15px; display:inline-block;"></span> Sehr viele Jobs
</div>
{% endmacro %}
"""

legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)


In [20]:
# Karte speichern
m.save("austria_heatmap.html")
